In [130]:
import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium
import contextily as ctx
from shapely.geometry import box
import xarray as xr
import matplotlib.animation as animation
import os
from branca.colormap import linear
from shapely.wkt import loads
from scipy.spatial import cKDTree
from sklearn.ensemble import RandomForestRegressor

In [131]:
# Setting
wdir = "/Users/weiha/Documents/bits/dades/CALIOPE/hourly/sconcno2/"
crs_latlon = "EPSG:4326"  # WGS84
crs_utm = "EPSG:32631"   # UTM zone 31

In [132]:
# Leer el archivo CSV con tipos específicos
ds = pd.read_csv(
    f"{wdir}NO2.csv",
    dtype={
        "sconcno2": float,  # Concentración de NO₂ como número
        "lat": float,       # Latitud como número
        "lon": float        # Longitud como número
    },
    low_memory=False  # Evitar el aviso de pandas sobre tipos mixtos
)

# Eliminar filas duplicadas basadas en la geometría
ds = ds.drop_duplicates(subset=["lat", "lon"])

# Renombrar sconcno2 a NO2
ds = ds.rename(columns={"sconcno2": "no2"})

# Convertir el DataFrame en un GeoDataFrame
geometry = [Point(xy) for xy in zip(ds['lon'], ds['lat'])]
gdf = gpd.GeoDataFrame(ds, crs=crs_utm, geometry=geometry)

# Display the first few rows of the CSV data
print(gdf.head())

         time        lat       lon       no2               geometry
0  2023-01-01  40.361107 -0.001190  0.000393  POINT (-0.001 40.361)
1  2023-01-01  40.360806  0.010620  0.000436   POINT (0.011 40.361)
2  2023-01-01  40.360510  0.022430  0.000485   POINT (0.022 40.361)
3  2023-01-01  40.360210  0.034241  0.000429    POINT (0.034 40.36)
4  2023-01-01  40.359894  0.046051  0.000415    POINT (0.046 40.36)


In [133]:
# Crear un DataFrame con las coordenadas proporcionadas
data = {'lat': [41.39216, 41.11588, 41.44398, 41.32177],
    'lon': [2.009802, 1.191975, 2.237875, 2.082141]}

df_target = pd.DataFrame(data)

# Convertir el DataFrame en un GeoDataFrame
geometry_target = [Point(xy) for xy in zip(df_target['lon'], df_target['lat'])]
gdf_target = gpd.GeoDataFrame(df_target, crs=crs_latlon, geometry=geometry_target)

# Mostrar el GeoDataFrame resultante
print(gdf_target)

        lat       lon                  geometry
0  41.39216  2.009802   POINT (2.0098 41.39216)
1  41.11588  1.191975  POINT (1.19198 41.11588)
2  41.44398  2.237875  POINT (2.23788 41.44398)
3  41.32177  2.082141  POINT (2.08214 41.32177)


In [134]:
# # Crear un colormap para los niveles de NO₂
# norm = mcolors.Normalize(vmin=gdf['no2'].min(), vmax=gdf['no2'].max())  # Normalizar valores
# cmap = plt.cm.viridis  # Usar el colormap 'viridis'

# # Centro del mapa basado en las coordenadas promedio
# map_center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
# m = folium.Map(location=map_center, zoom_start=10)  # Ajusta el zoom según la región

# # Añadir puntos al mapa con colores basados en niveles de NO₂
# for _, row in gdf.iterrows():
#     # Obtener el color para el nivel de NO₂
#     color = mcolors.to_hex(cmap(norm(row['no2'])))  # Convertir a formato hexadecimal
    
#     folium.CircleMarker(
#         location=[row.geometry.y, row.geometry.x],  # Latitud y longitud
#         radius=5,  # Tamaño del marcador
#         color=color,  # Color del borde
#         fill=True,
#         fill_color=color,  # Color de relleno
#         fill_opacity=0.7,
#         popup=f"NO₂: {row['no2']:.6f}"  # Mostrar el valor de NO₂
#     ).add_to(m)

# # Añadir una barra de colores (leyenda) usando branca
# color_scale = linear.viridis.scale(gdf['no2'].min(), gdf['no2'].max())
# color_scale.caption = 'NO₂ Concentration'
# color_scale.add_to(m)

# # Mostrar el mapa interactivo
# m

In [135]:
# Pick n random points for variogram fitting
n = 10000

np.random.seed(1)
pick_random_points = np.random.choice(len(gdf), size=min(n, len(gdf)), replace=False)
input_data_example = gdf.iloc[pick_random_points]

In [136]:
# ok = OrdinaryKriging(
#     input_data_example.geometry.x, input_data_example.geometry.y, input_data_example['no2'].values,
#     variogram_model='gaussian',
#     verbose=True, 
#     enable_plotting=False,
#     nlags=25
# )

# # Perform the interpolation
# z, ss = ok.execute('points', gdf_target.geometry.x, gdf_target.geometry.y)

# # Add results to the observation locations GeoDataFrame
# gdf_target['kriging_prediction'] = z
# gdf_target['kriging_variance'] = ss

# gdf_target_webmercator = gdf_target.to_crs(crs_utm)

# # Plotting the Kriging results
# fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# gdf_target.plot(column='kriging_prediction', ax=ax, legend=True, cmap='viridis', edgecolor='k')

# # Add contextily basemap (Web Mercator)
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# # Add title and labels
# ax.set_title('Interpolated Pollutant Concentrations (Kriging Results)')
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')

# # Show the plot
# plt.show()

In [137]:
# # Create a copy and convert to the correct CRS for folium (WGS84)
# gdf_target_wgs84 = gdf_target.copy().to_crs(crs_latlon)

# # Extract lat and lon for folium
# gdf_target_wgs84['lon'] = gdf_target_wgs84.geometry.x
# gdf_target_wgs84['lat'] = gdf_target_wgs84.geometry.y

# # Create a colormap
# norm = mcolors.Normalize(vmin=gdf_target_wgs84['kriging_prediction'].min(), vmax=gdf_target_wgs84['kriging_prediction'].max())
# cmap = plt.cm.viridis

# # Create a folium map centered around the data
# map_center = [gdf_target_wgs84['lat'].mean(), gdf_target_wgs84['lon'].mean()]
# m = folium.Map(location=map_center, zoom_start=12)

# # Add circle markers for each observation location with color based on kriging prediction
# for idx, row in gdf_target_wgs84.iterrows():
#     color = mcolors.to_hex(cmap(norm(row['kriging_prediction'])))
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=5,
#         color=color,
#         fill=True,
#         fill_color=color,
#         fill_opacity=0.7,
#         popup=f"NO2: {row['kriging_prediction']:.2f}"
#     ).add_to(m)

# # Save the map to an HTML file and display it
# #m.save('map.html')
# m


In [138]:
# Load necessary datasets
sp_model = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/sp_rawmodel_20220101.geojson")
sp_obs_locations = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/sp_obs_locations.geojson")
df_obs = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/obs_20220101.geojson")
domain_polygon = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/def_domini_bcn.geojson")
refined_grid = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/def_refined_grid_sf_100.geojson")
traffic_buffers = gpd.read_file("/Users/weiha/Documents/bits/dades/Hands-on_Downscaling/Data_python/traffic_buffers_100_class_OTM.geojson")
sp_obs_locations['geometry'] = gpd.points_from_xy(sp_obs_locations['lon'], sp_obs_locations['lat'], crs=crs_utm)

In [139]:
minx, miny, maxx, maxy = domain_polygon.total_bounds

# Crear la cuadrícula de 150m x 150m
cell_size = 500  # Tamaño de cada celda en metros
x_coords = np.arange(minx, maxx, cell_size)
y_coords = np.arange(miny, maxy, cell_size)

grid_cells = [box(x, y, x + cell_size, y + cell_size) for x in x_coords for y in y_coords]
grid = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=crs_utm)

# Recortar la cuadrícula al área de Barcelona
grid_bcn = gpd.overlay(grid, domain_polygon, how="intersection")

# Convertir a WGS84 para usar en Folium
grid_bcn_wgs84 = grid_bcn.to_crs(epsg=4326)

# Calcular los puntos centrales de las celdas (centroides)
grid_bcn_wgs84["centroid"] = grid_bcn_wgs84.geometry.centroid

# # Crear un mapa centrado en Barcelona
# map_center = [
#     grid_bcn_wgs84.geometry.centroid.y.mean(),
#     grid_bcn_wgs84.geometry.centroid.x.mean()
# ]
# m = folium.Map(location=map_center, zoom_start=12)

# # Añadir las celdas de la cuadrícula al mapa
# for _, row in grid_bcn_wgs84.iterrows():
#     folium.GeoJson(
#         row.geometry,
#         style_function=lambda x: {
#             "fillColor": "blue",
#             "color": "black",
#             "weight": 0.5,
#             "fillOpacity": 0.2,
#         },
#     ).add_to(m)

# # Añadir los puntos de la cota de malla (centroides)
# for _, row in grid_bcn_wgs84.iterrows():
#     folium.CircleMarker(
#         location=[row.centroid.y, row.centroid.x],  # Coordenadas del centro
#         radius=2,  # Tamaño del marcador
#         color="red",
#         fill=True,
#         fill_color="red",
#         fill_opacity=0.8,
#         popup=f"Centroid: ({row.centroid.y:.6f}, {row.centroid.x:.6f})"
#     ).add_to(m)

# # Mostrar el mapa interactivo
# m

C:\Users\weiha\AppData\Local\Temp\ipykernel_10840\284222150.py:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:32631
Right CRS: EPSG:4326

  grid_bcn = gpd.overlay(grid, domain_polygon, how="intersection")
C:\Users\weiha\AppData\Local\Temp\ipykernel_10840\284222150.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_bcn_wgs84["centroid"] = grid_bcn_wgs84.geometry.centroid


In [140]:
# Crear un nuevo GeoDataFrame a partir de grid_bcn_wgs84["geometry"]
grid_gdf = gpd.GeoDataFrame(geometry=grid_bcn_wgs84["centroid"], crs=crs_latlon)
centroid_list = list(grid_gdf["geometry"])
# Mostrar las primeras filas
print(centroid_list)

[<POINT (2.055 41.319)>, <POINT (2.055 41.323)>, <POINT (2.055 41.327)>, <POINT (2.055 41.332)>, <POINT (2.055 41.336)>, <POINT (2.055 41.341)>, <POINT (2.055 41.345)>, <POINT (2.055 41.35)>, <POINT (2.055 41.354)>, <POINT (2.055 41.359)>, <POINT (2.055 41.363)>, <POINT (2.055 41.368)>, <POINT (2.055 41.372)>, <POINT (2.055 41.377)>, <POINT (2.055 41.381)>, <POINT (2.055 41.386)>, <POINT (2.055 41.39)>, <POINT (2.055 41.395)>, <POINT (2.055 41.399)>, <POINT (2.055 41.404)>, <POINT (2.055 41.408)>, <POINT (2.055 41.413)>, <POINT (2.055 41.417)>, <POINT (2.055 41.422)>, <POINT (2.055 41.426)>, <POINT (2.054 41.431)>, <POINT (2.054 41.435)>, <POINT (2.054 41.44)>, <POINT (2.054 41.444)>, <POINT (2.054 41.449)>, <POINT (2.054 41.453)>, <POINT (2.054 41.458)>, <POINT (2.054 41.462)>, <POINT (2.054 41.466)>, <POINT (2.061 41.319)>, <POINT (2.061 41.323)>, <POINT (2.061 41.327)>, <POINT (2.061 41.332)>, <POINT (2.061 41.336)>, <POINT (2.061 41.341)>, <POINT (2.061 41.345)>, <POINT (2.061 41.3

In [141]:
# ok = OrdinaryKriging(
#     input_data_example.geometry.x, input_data_example.geometry.y, input_data_example['no2'].values,
#     variogram_model='gaussian',
#     verbose=True, 
#     enable_plotting=False,
#     nlags=25
# )

# # Perform the interpolation
# z, ss = ok.execute('points', grid_gdf.geometry.x, grid_gdf.geometry.y)

# # Add results to the observation locations GeoDataFrame
# grid_gdf['kriging_prediction'] = z
# grid_gdf['kriging_variance'] = ss

# grid_gdf_webmercator = grid_gdf.to_crs(crs_utm)

# # Plotting the Kriging results
# fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# grid_gdf.plot(column='kriging_prediction', ax=ax, legend=True, cmap='viridis', edgecolor='k')

# # Add contextily basemap (Web Mercator)
# ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# # Add title and labels
# ax.set_title('Interpolated NO2 Concentrations (Kriging Results)')
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')

# # Show the plot
# plt.show()

In [142]:
df_no2 = pd.read_csv('/Users/weiha/Documents/bits/dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('/Users/weiha/Documents/bits/dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')


# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])

# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)

# Transpose the dataframe to have each station ID as a row and the Date as the column
df_no2_transposed = df_no2.set_index('Date').transpose().reset_index()

# Melt the dataframe to have a 'Value' column for each hour
df_no2_melted = pd.melt(df_no2_transposed, id_vars=['index'], var_name='Date', value_name='no2')

# Rename the 'index' column to 'Station'
df_no2_melted.rename(columns={'index': 'Station'}, inplace=True)

# Drop rows with NaN values
df_no2_melted.dropna(subset=['no2'], inplace=True)

# Join the melted dataframe with the station information dataframe
df_obs = pd.merge(df_no2_melted, df_estacions, left_on='Station', right_on='code', how='inner')

# Drop the 'code' column as it is redundant
df_obs.drop(columns=['code','type'], inplace=True)
df_obs['geometry'] = gpd.points_from_xy(df_obs['lon'], df_obs['lat'], crs=crs_utm)
df_caliope = pd.read_csv(f'{wdir}NO2.csv') 

df_caliope.rename(columns={'sconcno2': 'no2', 'time': 'Date'}, inplace=True)

df_caliope['geometry'] = gpd.points_from_xy(df_caliope['lon'], df_caliope['lat'], crs=crs_utm)

df_combined = pd.concat([df_obs, df_caliope], ignore_index=True)

# Convert df_combined to a GeoDataFrame
df_combined = gpd.GeoDataFrame(df_combined, geometry='geometry', crs=crs_utm)

# Agrupar por 'geometry' y calcular la media de 'no2'
df_combined_mean = df_combined.groupby('geometry')['no2'].mean().reset_index()

In [143]:
# Crear GeoDataFrame con puntos de predicción
obs_locations = gpd.GeoDataFrame(
    {'geometry': centroid_list},
    crs=crs_utm
)

# Extraer coordenadas y valores del dataset conocido
coords = np.array([(geom.x, geom.y) for geom in df_combined_mean.geometry])
values = df_combined_mean['no2'].values

# Crear el árbol KD
tree = cKDTree(coords)

# Función IDW
def idw(x, y, tree, coords, values, power=2):
    """
    Interpolación por inverso de la distancia (IDW).
    - x, y: Coordenadas de predicción
    - tree: KDTree construido con los datos conocidos
    - coords: Coordenadas conocidas
    - values: Valores conocidos
    - power: Peso de la distancia
    """
    # Encontrar distancias y vecinos
    distances, idx = tree.query(np.c_[x, y], k=len(coords))
    
    # Manejo de distancias 0 (puntos coincidentes)
    distances[distances == 0] = 1e-10  # Evitar división por cero
    
    # Calcular pesos inversos a las distancias
    weights = 1 / distances**power
    
    # Normalizar los pesos
    weights /= np.sum(weights, axis=1, keepdims=True)
    
    # Calcular valores interpolados
    interpolated_values = np.sum(weights * values[idx], axis=1)
    return interpolated_values

# Coordenadas de los puntos a predecir
prediction_coords = np.array([(geom.x, geom.y) for geom in obs_locations.geometry])
x_pred, y_pred = prediction_coords[:, 0], prediction_coords[:, 1]

# Aplicar IDW a los puntos
obs_locations['predicted_value'] = idw(x_pred, y_pred, tree, coords, values)

print(obs_locations)


                  geometry  predicted_value
0     POINT (2.055 41.319)         0.141991
1     POINT (2.055 41.323)         0.168640
2     POINT (2.055 41.327)         0.159588
3     POINT (2.055 41.332)         0.173953
4     POINT (2.055 41.336)         0.168453
...                    ...              ...
1015   POINT (2.227 41.45)         0.492095
1016  POINT (2.227 41.454)         0.317388
1017  POINT (2.227 41.459)         0.366435
1018  POINT (2.227 41.463)         0.178502
1019  POINT (2.227 41.467)         0.258181

[1020 rows x 2 columns]


In [ ]:
# Crear un rango de colores basado en predicted_value
colormap = linear.viridis.scale(
    obs_locations['predicted_value'].min(),
    obs_locations['predicted_value'].max()
)
colormap.caption = "NO₂ Predicted Concentration"

# Crear un mapa centrado en las coordenadas promedio de los puntos
map_center = [
    obs_locations.geometry.y.mean(),
    obs_locations.geometry.x.mean()
]
m = folium.Map(location=map_center, zoom_start=12)

# Agregar los puntos al mapa
for _, row in obs_locations.iterrows():
    color = colormap(row['predicted_value'])  # Obtener el color para el valor
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],  # Coordenadas del punto
        radius=5,  # Tamaño del marcador
        color=color,  # Color del borde
        fill=True,
        fill_color=color,  # Color del relleno
        fill_opacity=0.7,
        popup=f"NO₂: {row['predicted_value']:.2f}"  # Información desplegable
    ).add_to(m)

# Agregar la barra de colores al mapa
colormap.add_to(m)

# Mostrar el mapa
m

In [144]:
# Cargar los datos

data = df_combined.sample(n=10000, random_state=42)
data.rename(columns={'Date': 'date', 'no2': 'concentration'}, inplace=True)
# Convert 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'], errors='coerce')
data.dropna(subset=['date'], inplace=True)
# Generar características adicionales (temporales y geográficas)
data['hour'] = data['date'].dt.hour
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month

# Variables predictoras y objetivo
X = data[['concentration', 'lat', 'lon', 'hour', 'day', 'month']]
y = data['concentration'].shift(-1)  # El siguiente valor de concentración

# Eliminar valores nulos generados por el shift
X = X[:-1]
y = y[:-1]

# Entrenar el modelo con todos los datos
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Predicción con un nuevo conjunto de datos
# Sustituye estos valores por los datos de entrada para la predicción
new_data = pd.DataFrame({
    'concentration': [0.46611],  # Concentración inicial
    'lat': [41.39216],
    'lon': [2.009802],
    'hour': [0],
    'day': [1],
    'month': [1]
})
new_concentration = model.predict(new_data)
print(f"Nueva concentración: {new_concentration[0]}")

Nueva concentración: 0.5532452156197503


In [145]:
obs_loc = obs_locations.copy()
obs_loc['lat'] = obs_loc.geometry.y
obs_loc['lon'] = obs_loc.geometry.x
obs_loc = obs_loc.drop(columns=['geometry'])
obs_loc.rename(columns={'predicted_value': 'concentration'}, inplace=True)

# Generate a date range from January 1, 2023 to December 31, 2023, hourly
date_range = pd.date_range(start='2023-01-01 00:00:00', end='2023-12-31 23:00:00', freq='H')

# Repeat the date range for each point
obs_loc = obs_loc.loc[obs_loc.index.repeat(len(date_range))].reset_index(drop=True)

# Assign the repeated date range to the 'Date' column
obs_loc['date'] = pd.concat([pd.Series(date_range)] * 4, ignore_index=True)

# Add an ID column
obs_loc['id'] = obs_loc.index + 1

print(obs_loc.head())
# Fit the model for each row in obs_loc
obs_loc['predicted_concentration'] = obs_loc.apply(
    lambda row: model.predict(pd.DataFrame({
        'concentration': [row['concentration']],
        'lat': [row['lat']],
        'lon': [row['lon']],
        'hour': [row['date'].hour],
        'day': [row['date'].day],
        'month': [row['date'].month]
    }))[0], axis=1
)

print(obs_loc.head())

C:\Users\weiha\AppData\Local\Temp\ipykernel_10840\379389475.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start='2023-01-01 00:00:00', end='2023-12-31 23:00:00', freq='H')


   concentration        lat       lon                date  id
0       0.141991  41.318701  2.055282 2023-01-01 00:00:00   1
1       0.141991  41.318701  2.055282 2023-01-01 01:00:00   2
2       0.141991  41.318701  2.055282 2023-01-01 02:00:00   3
3       0.141991  41.318701  2.055282 2023-01-01 03:00:00   4
4       0.141991  41.318701  2.055282 2023-01-01 04:00:00   5


KeyboardInterrupt: 